In [3]:
import cv2
import mediapipe as mp
import os
import csv

In [2]:
landmark_names = [
        "WRIST",
        "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
        "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
        "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
        "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
        "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
    ]

In [2]:
folder_path ="/home/ruwantha/Repos/sign-project/SignLanguageTutor/Data/SinhalaData/ImageSet/Sinhala Images/අ - 06"
output_csv_path = "/home/ruwantha/Repos/sign-project/SignLanguageTutor/Data/SinhalaData/CSV/test.csv"

In [4]:
# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands

In [31]:
os.path.exists(output_csv_path)

True

In [39]:
# Create or overwrite the CSV file and write the header
with open(output_csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header with columns for each landmark's coordinates for both hands
    header = ["Image"]
    for hand in ["Left", "Right"]:
        for landmark_name in landmark_names:
            header.extend([f"{hand}_{landmark_name}_X", f"{hand}_{landmark_name}_Y", f"{hand}_{landmark_name}_Z"])
    csv_writer.writerow(header)

    # Initialize the Hand model
    with mp_hands.Hands(
            static_image_mode=True,  # Set to static mode for images
            max_num_hands=2,  # Detect up to 2 hands
            min_detection_confidence=0.5  # Detection threshold
    ) as hands:
        # Loop through all files in the folder
        for filename in os.listdir(folder_path):
            # Check if file is an image (e.g., .jpg, .jpeg, .png)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)

                # Convert the image to RGB for MediaPipe processing
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Process the image and detect hands
                result = hands.process(rgb_image)

                # Initialize row data with the filename
                row_data = [filename]

                # Prepare placeholders for left and right hand landmarks
                left_hand_data = [0.0] * (len(landmark_names) * 3)
                right_hand_data = [0.0] * (len(landmark_names) * 3)

                if result.multi_hand_landmarks and result.multi_handedness:
                    for hand_idx, (hand_landmarks, handedness) in enumerate(
                            zip(result.multi_hand_landmarks, result.multi_handedness)):
                        # Identify the hand as "Left" or "Right"
                        hand_label = handedness.classification[0].label
                        print(f"Handedness for {filename}: {handedness.classification[0].label}")


                        # Prepare data list for the detected hand
                        hand_data = []

                        # Get wrist coordinates for normalization
                        wrist = hand_landmarks.landmark[0]
                        wrist_x, wrist_y, wrist_z = wrist.x, wrist.y, wrist.z

                        # Add normalized landmark data for each landmark
                        for landmark in hand_landmarks.landmark:
                            norm_x = round(landmark.x - wrist_x,2)
                            norm_y = round(landmark.y - wrist_y,2)
                            norm_z = round(landmark.z - wrist_z,2)
                            hand_data.extend([norm_x, norm_y, norm_z])

                        # Assign data to left or right hand list based on label
                        if hand_label == "Left":
                            left_hand_data = hand_data
                        elif hand_label == "Right":
                            right_hand_data = hand_data

                # Append left and right hand data to the row data
                row_data.extend(left_hand_data)
                row_data.extend(right_hand_data)

                # Write data for this image to the CSV file
                csv_writer.writerow(row_data)

print(f"Landmark data saved to {output_csv_path}")


I0000 00:00:1733024504.514909   63808 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1733024504.517028   69931 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.7-arch1.1), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1733024504.532342   69923 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733024504.546269   69922 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Handedness for 2.jpg: Left
Handedness for 21.jpg: Left
Handedness for 45.jpg: Left
Handedness for 44.jpg: Left
Handedness for 20.jpg: Left
Handedness for 1.jpg: Left
Landmark data saved to /home/ruwantha/Repos/sign-project/SignLanguageTutor/Data/SinhalaData/CSV/test.csv


In [5]:
import os
import cv2
import mediapipe as mp
import csv


landmark_names = [  # Define landmark names
    "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
    "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
    "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
    "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
    "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
]

# Create or overwrite the CSV file and write the header
with open(output_csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header with columns for the right hand landmarks
    header = ["Image"]
    for landmark_name in landmark_names:
        header.extend([f"Right_{landmark_name}_X", f"Right_{landmark_name}_Y", f"Right_{landmark_name}_Z"])
    csv_writer.writerow(header)

    # Initialize the Hand model
    with mp.solutions.hands.Hands(
            static_image_mode=True,  # Static mode for images
            max_num_hands=2,  # Detect up to 2 hands
            min_detection_confidence=0.5  # Detection threshold
    ) as hands:
        # Loop through all files in the folder
        for filename in os.listdir(folder_path):
            # Check if file is an image (e.g., .jpg, .jpeg, .png)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)

                # Convert the image to RGB for MediaPipe processing
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Process the image and detect hands
                result = hands.process(rgb_image)

                # Initialize row data with the filename
                row_data = [filename]

                # Prepare placeholder for right-hand landmarks
                right_hand_data = [0.0] * (len(landmark_names) * 3)

                if result.multi_hand_landmarks and result.multi_handedness:
                    for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                        # Identify the hand as "Right"
                        hand_label = handedness.classification[0].label
                        if hand_label == "Right":
                            print(f"Right hand detected for {filename}")

                            # Prepare data list for the detected right hand
                            hand_data = []

                            # Get wrist coordinates for normalization
                            wrist = hand_landmarks.landmark[0]
                            wrist_x, wrist_y, wrist_z = wrist.x, wrist.y, wrist.z

                            # Add normalized landmark data for each landmark
                            for landmark in hand_landmarks.landmark:
                                norm_x = round(landmark.x - wrist_x, 2)
                                norm_y = round(landmark.y - wrist_y, 2)
                                norm_z = round(landmark.z - wrist_z, 2)
                                hand_data.extend([norm_x, norm_y, norm_z])

                            # Assign data to right-hand list
                            right_hand_data = hand_data
                            break  # Stop processing other hands since we're only interested in the right hand

                # Append right-hand data to the row data
                row_data.extend(right_hand_data)

                # Write data for this image to the CSV file
                csv_writer.writerow(row_data)

print(f"Right hand landmark data saved to {output_csv_path}")


I0000 00:00:1733024982.705546   70711 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1733024982.709369   70944 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.7-arch1.1), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733024982.734094   70935 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733024982.761164   70936 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733024982.839755   70934 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Right hand landmark data saved to /home/ruwantha/Repos/sign-project/SignLanguageTutor/Data/SinhalaData/CSV/test.csv
